In [11]:
import pyscf
import ffsim
import numpy as np
from opt_einsum import contract

In [12]:
# Build N2 molecule
mol = pyscf.gto.Mole()
mol.build(
    atom=[["N", (0, 0, 0)], ["N", (1.0, 0, 0)]],
    basis="sto-6g",
    # basis="6-31g",
    symmetry="Dooh",
)

# Define active space
n_frozen = 2
active_space = range(n_frozen, mol.nao_nr())

# Get molecular data and Hamiltonian
scf = pyscf.scf.RHF(mol).run()
mol_data = ffsim.MolecularData.from_scf(scf, active_space=active_space)
norb, nelec = mol_data.norb, mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian
print(f"norb = {norb}")
print(f"nelec = {nelec}")

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = pyscf.cc.CCSD(
    scf, frozen=[i for i in range(mol.nao_nr()) if i not in active_space]
).run()

# Use 2 ansatz layers
n_reps = 2
# Use interactions implementable on a square lattice
pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
ucj_op = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
    ccsd.t2, t1=ccsd.t1, n_reps=n_reps, interaction_pairs=(pairs_aa, pairs_ab)
)

converged SCF energy = -108.464957764796
norb = 8
nelec = (5, 5)
E(CCSD) = -108.5933309085008  E_corr = -0.1283731437052354


In [13]:
nocc, _, nvrt, _ = ccsd.t2.shape
ccsd.t2.shape

(5, 5, 3, 3)

In [14]:
diag_coulomb_mats, orbital_rotations = ffsim.linalg.double_factorized_t2(ccsd.t2, tol=1e-8)

In [15]:
diag_coulomb_mats.shape

(15, 2, 8, 8)

In [16]:
orbital_rotations.shape

(15, 2, 8, 8)

In [17]:
n_reps = 1

In [22]:
reconstructed = (
        1j
        * contract(
            "mkpq,mkap,mkip,mkbq,mkjq->ijab",
            diag_coulomb_mats[:n_reps],
            orbital_rotations[:n_reps],
            orbital_rotations[:n_reps].conj(),
            orbital_rotations[:n_reps],
            orbital_rotations[:n_reps].conj(),
            # optimize="greedy"
        )[:nocc, :nocc, nocc:, nocc:]
    )

reconstructed.shape

(5, 5, 3, 3)

In [19]:
diff = reconstructed - ccsd.t2

In [20]:
np.max(np.abs(diff))

np.float64(0.052682008717937295)